In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
OPENAI_API_KEY=os.environ["OPENAI_API_KEY"]

In [3]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

Experiemnts 1: Use regular Sequential Chains

In [4]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain

In [5]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [6]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to {lang1}:"
    "\n\n{review}"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="Translation1")

/var/folders/rm/ffmtyr9j47nccz_6s1wjrlzh0000gp/T/ipykernel_6472/2128832746.py:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="Translation1")


In [7]:
second_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to {lang2}:"
    "\n\n{Translation1}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="Translation2")


In [8]:
third_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to {lang3}:"
    "\n\n{Translation2}"
)
chain_three = LLMChain(llm=llm, prompt=third_prompt,  output_key="Translation3")

In [9]:
fourth_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to {lang4}:"
    "\n\n{Translation3}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,  output_key="Translation4")

In [10]:
fifth_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to {lang0}:"
    "\n\n{Translation4}"
)
chain_five = LLMChain(llm=llm, prompt=fifth_prompt,  output_key="Translation5")

In [11]:
print(first_prompt)
print(second_prompt)
print(third_prompt)
print(fourth_prompt)
print(fifth_prompt)


input_variables=['lang1', 'review'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['lang1', 'review'], input_types={}, partial_variables={}, template='Translate the following review to {lang1}:\n\n{review}'), additional_kwargs={})]
input_variables=['Translation1', 'lang2'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['Translation1', 'lang2'], input_types={}, partial_variables={}, template='Translate the following review to {lang2}:\n\n{Translation1}'), additional_kwargs={})]
input_variables=['Translation2', 'lang3'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['Translation2', 'lang3'], input_types={}, partial_variables={}, template='Translate the following review to {lang3}:\n\n{Translation2}'), additional_kwargs={})]
input_variables=['Translation3', 'lang4'] input_types={} partial_variable

In [12]:
### DEfine the Sequential Chains
overall_chain = SequentialChain(
    chains=[chain_one,chain_two,chain_three,chain_four,chain_five],
    input_variables=['review','lang0','lang1','lang2','lang3','lang4'],
    output_variables=['Translation1','Translation2','Translation3','Translation4','Translation5'],
    verbose=True
)

In [13]:
### Substantiate the Sequentia chain.
inputVar={'review':'I love my tennis lancher very much. It is compact, powerful and works just as I expect. Definitely recommend it!', 
           'lang0':'English','lang1': 'French', 'lang2':'Germany','lang3':'Italian','lang4':'Chinese'}

tt = overall_chain(inputVar)


/var/folders/rm/ffmtyr9j47nccz_6s1wjrlzh0000gp/T/ipykernel_6472/813702941.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  tt = overall_chain(inputVar)
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.


In [14]:
print("i =  0 ----   ",tt['review'])
for i in range(1,6):
    print("i = ",i," ----", tt['Translation'+str(i)])


i =  0 ----    I love my tennis lancher very much. It is compact, powerful and works just as I expect. Definitely recommend it!
i =  1  ---- J'adore mon lanceur de tennis. Il est compact, puissant et fonctionne exactement comme je m'y attends. Je le recommande vivement !
i =  2  ---- Ich liebe meinen Tennisschläger. Er ist kompakt, leistungsstark und funktioniert genau so, wie ich es erwarte. Ich kann ihn nur wärmstens empfehlen!
i =  3  ---- Adoro la mia racchetta da tennis. È compatta, potente e funziona esattamente come mi aspetto. Posso solo raccomandarla caldamente!
i =  4  ---- 我喜欢我的网球拍。它紧凑，强大，而且正如我期望的那样工作。我强烈推荐它！
i =  5  ---- I love my tennis racket. It is compact, powerful, and works just as I expected. I highly recommend it!
